In [1]:
import os
import numpy as np
import pandas as pd
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.nist_score import sentence_nist
from Levenshtein import distance as levenshtein_distance

In [2]:
rouge = Rouge()

In [12]:
def calculate_bleu(row):
    from nltk.translate.bleu_score import SmoothingFunction
    smoothie = SmoothingFunction().method4
    return np.round(sentence_bleu([row['tokenized_gold_standard_nl']], row['tokenized_generated_nl'], smoothing_function=smoothie), 4)

def calculate_nist(row):
    try:
        return np.round(sentence_nist([row['tokenized_gold_standard_nl']], row['tokenized_generated_nl']), 4)
    except ZeroDivisionError:
        return 0.0

def calculate_rougeL(row):
    try:
        scores = rouge.get_scores(row['generated_nl'], row['gold_standard_nl'])
        return np.round(scores[0]['rouge-l']['f'], 4)
    except:
        return 0

def calculate_levenshtein(row):
    try:
        return np.round(levenshtein_distance(row['generated_nl'], row['gold_standard_nl']), 4)
    except:
        return len(row['gold_standard_nl'])

def tokenize(x):
    try:
        return word_tokenize(str(x))
    except:
        return str(x).split()
# from bert_score import score

# def calculate_bert_score(row):
    # P, R, F1 = score(row['generated_nl'], row['gold_standard_nl'], lang="en", verbose=True)
    # return F1.mean()

In [13]:
model = "mistral-7b-zeroshot-finetuned"

for file in os.listdir(f"../data/alpe-experiments/raw/{model}/"):
    df = pd.read_csv(f"../data/alpe-experiments/raw/{model}/{file}", sep='\t')
    print(file)
    df['tokenized_generated_nl'] = df['generated_nl'].apply(tokenize)
    df['tokenized_gold_standard_nl'] = df['gold_standard_nl'].apply(word_tokenize)
    df['sentence_bleu'] = df.apply(calculate_bleu, axis=1).astype(float)
    df['sentence_nist'] = df.apply(calculate_nist, axis=1).astype(float)
    df['rougeL'] = df.apply(calculate_rougeL, axis=1).astype(float)
    df['levenshtein'] = df.apply(calculate_levenshtein, axis=1).astype(float)

    df.to_csv(f"../data/alpe-experiments/metrics/{model}/{file}", sep='\t', index=False)

qald_9_plus_test_wikidata.json_en.tsv
qald_9_plus_train_wikidata.json_en.tsv
qald_9_plus_test_wikidata.json_de.tsv
qald_9_plus_train_wikidata.json_ru.tsv
